In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import urllib
from operator import itemgetter

import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("info")

DEBUG:root:info


In [94]:
def get_beautiful_soup_of_url(url):
    opener = urllib.request.build_opener()
    opener.addheaders = [('User-Agent', 'Mozilla/5.0')]
    try:
        loc_doc = opener.open(url)
        soup = BeautifulSoup(loc_doc, 'html.parser')
        return soup
    except:
        return ''

def get_area_link(main_area):
    return str(main_area).split("\n")[4].split(" ")[3].replace('href="','').replace('"','')

def get_area_name(main_area):
    return get_area_link(main_area).split("/")[-1].replace('"','')

def get_area_dict(main_area):
    return {"name":get_area_name(main_area), "link":get_area_link(main_area)}

In [3]:
mp_url = "https://www.mountainproject.com"
url_soup = get_beautiful_soup_of_url(mp_url)
route_guide = url_soup.find("div", {"id": "route-guide"})

area_data = [get_area_dict(main_area) for main_area in route_guide.find_all("strong")]
area_data = {v['name']:v for v in area_data}.values()
area_data = sorted(area_data, key=itemgetter('name'))

In [4]:
def get_sub_area_link(left_nav_data):
    return str(left_nav_data).split("\n")[1].split('href="')[1].split('"')[0]

def get_sub_area_name(left_nav_data):
    return get_sub_area_link(left_nav_data).split("/")[-1]

def get_sub_area_dict(left_nav_data):
    return {"name":get_sub_area_name(left_nav_data), "link":get_sub_area_link(left_nav_data)}

def get_sub_area_data(sub_area_data):
    if sub_area_data['link']:
        sub_area_soup = get_beautiful_soup_of_url(sub_area_data["link"])
        sub_area_data = [get_sub_area_dict(left_nav_data) for left_nav_data in sub_area_soup.find_all("div", {"class": "lef-nav-row"})]
        sub_area_data = {v['name']:v for v in sub_area_data}.values()
        sub_area_data = sorted(sub_area_data, key=itemgetter('name'))
        return sub_area_data
    else:
        return []

In [5]:
def return_sub_area_links(area_data):
    temp_list = get_sub_area_data(area_data)
    all_data = []
    for area in temp_list:
        all_data+=return_sub_area_links(area)
    return all_data+temp_list

In [87]:
def get_hierarchy_from_link(areas):
    climbing_area_html = get_beautiful_soup_of_url(areas)
    try:
        area_hierarchy_html = climbing_area_html.find("div", {"class": "mb-half small text-warm"})
        area_hierarchy = [str(area)[2:].split('"')[0] for area in str(area_hierarchy_html).split('href')[2:]]
        return area_hierarchy
    except:
        return []
    
def get_routes_from_link(areas):
    climbing_area_html = get_beautiful_soup_of_url(areas)
    try:
        area_hierarchy_html = climbing_area_html.find("div", {"class": "max-height max-height-md-0 max-height-xs-150"})
        area_hierarchy = [str(area)[2:].split('"')[0] for area in str(area_hierarchy_html) \
                          .split('href')[1:len(str(area_hierarchy_html) \
                          .split('href'))-1]]
        return area_hierarchy
    except:
        return []

In [ ]:
for areas in area_data:
    sub_area_master=return_sub_area_links(areas)
    all_area_data_df = pd.DataFrame(sub_area_master)
    all_area_data_df['hierarchy'] = all_area_data_df['link'].map(get_hierarchy_from_link)
    all_area_data_df['route_links'] = all_area_data_df['link'].map(get_routes_from_link)
    all_area_data_df.to_csv('data/'+areas['name']+'_climbing_areas.csv')

In [95]:
for areas in area_data:
    all_area_data_df = pd.read_csv(areas['name']+'_climbing_areas.csv')
    all_area_data_df['hierarchy'] = all_area_data_df['link'].map(get_hierarchy_from_link)
    all_area_data_df['route_links'] = all_area_data_df['link'].map(get_routes_from_link)
    all_area_data_df.to_csv('data/'+areas['name']+'_climbing_areas.csv')

In [109]:
import ast
texas_climbing = pd.read_csv('data/texas_climbing_areas.csv', index_col=None)

In [110]:
x = ast.literal_eval(texas_climbing.iloc[0][5])
set_of_routes = [n.strip() for n in x]

In [111]:
set_of_routes

['https://www.mountainproject.com/route/112806790/aretebulge',
 'https://www.mountainproject.com/route/112134969/flash-crack',
 'https://www.mountainproject.com/route/112806412/stand-off',
 'https://www.mountainproject.com/route/106085170/thin-crack',
 'https://www.mountainproject.com/route/106085185/touch-of-class']